In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.optimize

import pytz

import trfp

central_time = pytz.timezone('US/Central')

sns.set(style="darkgrid")
sns.set(rc={'figure.figsize':(7,4.33)})


In [ ]:
tr3997 = trfp.TrolleyRun(3997)
# tr3997.interp_data.to_csv("./run3997.csv")



# moment_df = tr3997.moment_dataframe()

In [ ]:
# 
st_to_plot = 'st30,m1'
sns.relplot(x='tr_phi', y=st_to_plot, data = moment_df, kind='line')
plt.show()

In [ ]:
# time to fit the stations

def trolley_effect(x, B0, A, x0):
    # x0 needs to be in deg, not mm
    # 2*pi*7112/360 mm per deg
    deg_per_mm = 360./(2.*np.pi*7112.)
    return (B0 + A * ((131./204.)*np.exp(-(x-x0-153*deg_per_mm)**2/(2*25.6*deg_per_mm)**2)
                      + np.exp(-(x-x0-266*deg_per_mm)**2/(2*25.7*deg_per_mm)**2)
                      + np.exp(-(x-x0-348*deg_per_mm)**2/(2*25*deg_per_mm)**2)))

station_ring_phi = np.zeros(72)

station_to_plot = 30

for st in np.arange(72):
    label = 'st'+str(st)+',m1'

    x = moment_df['tr_phi']
    y = moment_df[label]

    p0 = (np.median(y),np.max(y)-np.median(y),trfp.STATION_RING_PHI[st])

    bounds = ((np.min(y), 0.5*(np.max(y)-np.min(y)), trfp.STATION_RING_PHI[st]-10),
              (np.max(y), 1.5*(np.max(y)-np.min(y)), trfp.STATION_RING_PHI[st]+10))
    
    (coeffs, covars) = scipy.optimize.curve_fit(trolley_effect, x, y, p0=p0, bounds=bounds)
    
    station_ring_phi[st] = coeffs[2]
    
    if st == station_to_plot:
        y_fit = trolley_effect(x,coeffs[0],coeffs[1],coeffs[2])
        plt.plot(x,y,'.',x,y_fit)
        plt.xlim(coeffs[2]-5,coeffs[2]+5)
        plt.show()

In [ ]:
print station_ring_phi

In [ ]:
print station_ring_phi
print '\n'
print station_ring_phi-trfp.STATION_RING_PHI

In [ ]:
x1 = np.array([350.17685631, 354.30406708, 358.83623806, 4.35240544, 9.32206272
, 14.20303035, 20.20399984, 23.31454619, 29.32704948, 34.33090375
, 39.33314149, 43.31186448, 50.20790264, 53.31372932, 59.34344047
, 64.31571127, 69.3447512, 73.29678754, 80.15567847, 83.32465393
, 89.30462637, 94.312144, 99.30329035, 103.30144532, 110.19012489
, 113.31473486, 119.33139431, 124.31805867, 129.30248023, 133.26492222
, 140.20506912, 143.37866634, 149.35650273, 154.33030655, 159.35134591
, 163.30280959, 170.17258808, 173.37578725, 179.37086803, 184.34860116
, 189.35874727, 193.31203033, 200.17397531, 203.38743279, 209.34640381
, 214.31348996, 219.33820433, 223.31708302, 230.22539407, 233.41646226
, 239.35416455, 244.33504266, 249.3766332, 253.34497409, 260.21109934
, 263.46900851, 269.35210197, 274.34318626, 279.3735428, 283.41452811
, 290.17557338, 293.37088886, 299.33322785, 304.32034054, 309.33729456
, 313.29371942, 320.18673347, 323.41222932, 329.32848823, 334.34625774
, 339.34548139, 343.40617345])

x2 = np.array([350.17240943, 354.34820922, 358.83396603, 4.33699416, 9.32859981
, 14.25245337, 20.20486029, 23.32400696, 29.32688629, 34.33315183
, 39.33644653, 43.30444831, 50.19159463, 53.31358289, 59.3287227
, 64.34107629, 69.33861661, 73.3023333, 80.18108822, 83.31710966
, 89.31049552, 94.34376604, 99.31746476, 103.29782323, 110.18852972
, 113.30260946, 119.33934967, 124.32733139, 129.30398564, 133.27274034
, 140.20983136, 143.37555541, 149.32822406, 154.32833527, 159.36288459
, 163.31693923, 170.2095447, 173.39884053, 179.35802611, 184.34843557
, 189.35584984, 193.31109049, 200.16749673, 203.38067579, 209.34030038
, 214.31431814, 219.33479797, 223.32315712, 230.2171529, 233.42389423
, 239.35826272, 244.33547348, 249.3739451, 253.33248402, 260.18351555
, 263.46357302, 269.35676154, 274.34243806, 279.37853235, 283.40381883
, 290.20307161, 293.37596783, 299.34781249, 304.29364539, 309.33899488
, 313.29212657, 320.1994366, 323.41448133, 329.32483805, 334.35134602
, 339.35314171, 343.41381479])

out = np.round((x1+x2)/2,2)